<a href="https://colab.research.google.com/github/priyal6/AI-AGENT/blob/main/crewai_for_financial_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install crewai crewai_tools==0.1.6 langchain_community==0.0.29 langchain_groq PyGithub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of crewai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of crewai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-groq to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-groq to determine which version is compatible with other requirements. This could take a while

In [4]:
import os

from crewai import Agent, Task, Crew, Process
from langchain_groq import ChatGroq
from google.colab import userdata

# Set API key for Groq
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [5]:
llm =ChatGroq(model_name = "llama3-70b-8192")
llm.invoke("Hi")

AIMessage(content="Hi! It's nice to meet you. Is there something I can help you with, or would you like to chat?", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 11, 'total_tokens': 37, 'completion_time': 0.07500302, 'prompt_time': 0.000149158, 'queue_time': 0.17099018, 'total_time': 0.075152178}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_de9fa14705', 'finish_reason': 'stop', 'logprobs': None}, id='run-991597c3-c47e-4a4b-a0a0-fc1eb62c675c-0')

In [6]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [8]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=llm # Pass the initialized ChatGroq model
)

In [10]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm = llm
)

In [11]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm = llm
)

In [12]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm = llm
)

In [13]:
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [14]:
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

In [15]:
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [16]:
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

In [17]:
from crewai import Crew, Process


# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent,
            trading_strategy_agent,
            execution_agent,
            risk_management_agent],

    tasks=[data_analysis_task,
           strategy_development_task,
           execution_planning_task,
           risk_assessment_task],

    manager_llm=ChatGroq(model_name = "llama3-70b-8192"),
    process=Process.hierarchical,
    verbose=True)

In [18]:

# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [19]:
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [2025-07-16 21:47:40][DEBUG]: Working Agent: Crew Manager
 [2025-07-16 21:47:40][INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
Thought: I need to understand the current market data for AAPL and identify trends and predict market movements. I'll delegate the task of gathering and analyzing market data to the Data Analyst.

Action: Delegate work to coworker
Action Input: {"task": "Gather and analyze market data for AAPL", "context": "Please collect the latest market data for AAPL, including historical prices, trading volumes, and any other relevant metrics. Analyze the data using statistical modeling and machine learning techniques to identify trends and patterns.", "coworker": "Data Analyst"}


> Entering new CrewAgentExecutor chain...
Thought: I need to gather the latest market data for AAPL from 

KeyboardInterrupt: 